In [ ]:
import sys, math, json
from datetime import datetime
from pathlib import Path

import cv2
import numpy as np
from PySide6 import QtCore, QtGui, QtWidgets
import mediapipe as mp

# -------- optional PDF ----------
try:
    from fpdf import FPDF
    _PDF_OK = True
except Exception:
    _PDF_OK = False

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

finger_joints = {
    "thumb": [1, 2, 3, 4],
    "index": [5, 6, 7, 8],
    "middle": [9, 10, 11, 12],
    "ring": [13, 14, 15, 16],
    "pinky": [17, 18, 19, 20]
}

# ---------- ring conversions ----------
def diameter_to_us_ring_size(mm):
    if mm is None: return None
    table = {
        14.0: 3, 14.4: 3.5, 14.8: 4, 15.2: 4.5, 15.6: 5,
        16.0: 5.5, 16.5: 6, 16.9: 6.5, 17.3: 7, 17.7: 7.5,
        18.1: 8, 18.5: 8.5, 19.0: 9, 19.4: 9.5, 19.8: 10,
        20.2: 10.5, 20.6: 11, 21.0: 11.5, 21.4: 12, 21.8: 12.5, 22.2: 13
    }
    closest = min(table.keys(), key=lambda x: abs(x - mm))
    return table[closest]

def diameter_to_eu_size(mm):
    if mm is None: return None
    return round((math.pi * mm) * 2) / 2.0  # circumference rounded to 0.5 mm

# ---------- helpers ----------
def ensure_dir(p): Path(p).mkdir(parents=True, exist_ok=True)
def stamp(): return datetime.now().strftime("%Y%m%d_%H%M%S")

def euclidean_distance(p1, p2, w, h):
    x1, y1 = int(p1.x * w), int(p1.y * h)
    x2, y2 = int(p2.x * w), int(p2.y * h)
    return math.hypot(x2 - x1, y2 - y1), (x1, y1), (x2, y2)

def save_json(output_dir, base, measurements, meta):
    ensure_dir(output_dir)
    path = Path(output_dir) / f"{base}.json"
    payload = {
        "timestamp": datetime.now().isoformat(timespec="seconds"),
        "scale_mm_per_pixel": meta.get("scale_mm_per_pixel"),
        "reference": {
            "reference_mm": meta.get("reference_mm"),
            "reference_pixels": meta.get("reference_pixels"),
            "eu_rounding_step_mm": 0.5
        },
        "image_path": meta.get("image_path"),
        "notes": meta.get("notes", []),
        "fingers": measurements,
    }
    with open(path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
    return str(path)

def save_pdf(output_dir, base, image_path, measurements, meta):
    if not _PDF_OK:
        print("[WARN] fpdf2 not installed. Run: pip install fpdf2")
        return None
    ensure_dir(output_dir)
    pdf_path = Path(output_dir) / f"{base}.pdf"
    pdf = FPDF(unit="mm", format="A4"); pdf.add_page(); pdf.set_auto_page_break(True, 15)
    pdf.set_font("Helvetica", "B", 16); pdf.cell(0, 10, "Hand & Ring Size Report", ln=True)
    pdf.set_font("Helvetica", "", 11)
    pdf.cell(0, 6, f"Timestamp: {datetime.now().isoformat(timespec='seconds')}", ln=True)
    pdf.cell(0, 6, f"Scale (mm/px): {meta.get('scale_mm_per_pixel') or 'N/A'}", ln=True)
    ref_mm, ref_px = meta.get("reference_mm"), meta.get("reference_pixels")
    pdf.cell(0, 6, f"Reference: {ref_mm} mm over {ref_px} px" if (ref_mm and ref_px) else "Reference: None", ln=True)
    pdf.ln(4); pdf.set_font("Helvetica", "B", 12); pdf.cell(0, 8, "Annotated Image:", ln=True)
    if image_path and Path(image_path).exists(): pdf.image(str(image_path), w=180)
    else: pdf.set_font("Helvetica", "I", 11); pdf.cell(0, 8, "(No image)", ln=True)
    pdf.ln(6); pdf.set_font("Helvetica", "B", 12)
    headers = ["Finger", "Span (px)", "Diameter (mm)", "US Size", "EU Size"]
    col_w = [36, 32, 36, 32, 32]
    for i, h in enumerate(headers): pdf.cell(col_w[i], 8, h, border=1, align="C")
    pdf.ln(8); pdf.set_font("Helvetica", "", 11)
    for finger, vals in measurements.items():
        px = vals.get("pixels"); mm = vals.get("diameter_mm")
        us = vals.get("us_ring_size"); eu = vals.get("eu_ring_size")
        pdf.cell(col_w[0], 8, finger.capitalize(), border=1)
        pdf.cell(col_w[1], 8, f"{px:.1f}" if isinstance(px,(int,float)) else "—", border=1, align="C")
        pdf.cell(col_w[2], 8, f"{mm:.2f}" if isinstance(mm,(int,float)) else "—", border=1, align="C")
        pdf.cell(col_w[3], 8, str(us) if us is not None else "N/A", border=1, align="C")
        pdf.cell(col_w[4], 8, f"{eu:.1f}" if isinstance(eu,(int,float)) else "N/A", border=1, align="C")
        pdf.ln(8)
    pdf.output(str(pdf_path)); return str(pdf_path)

def write_report_files(output_dir, annotated_bgr, measurements, meta):
    ensure_dir(output_dir); base = f"ring_report_{stamp()}"
    img_path = Path(output_dir) / f"{base}.png"
    cv2.imwrite(str(img_path), annotated_bgr)
    meta = {**meta, "image_path": str(img_path)}
    json_path = save_json(output_dir, base, measurements, meta)
    pdf_path = save_pdf(output_dir, base, str(img_path), measurements, meta)
    return {"json": json_path, "pdf": pdf_path, "image": str(img_path)}

# ---------- mask / composition ----------
def build_hand_mask(results, shape, dilate_px=12, feather_px=8):
    h, w = shape[:2]
    mask = np.zeros((h, w), dtype=np.uint8)
    if not results or not results.multi_hand_landmarks:
        return mask.astype(np.float32)/255.0
    for hand in results.multi_hand_landmarks:
        pts = np.array([[int(lm.x*w), int(lm.y*h)] for lm in hand.landmark], dtype=np.int32)
        hull = cv2.convexHull(pts)
        cv2.fillConvexPoly(mask, hull, 255)
    if dilate_px>0:
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*dilate_px+1, 2*dilate_px+1))
        mask = cv2.dilate(mask, k, 1)
    if feather_px>0:
        mask = cv2.GaussianBlur(mask, (2*feather_px+1, 2*feather_px+1), 0)
    return mask.astype(np.float32)/255.0

def apply_hand_focus_effect(frame_bgr, results, mode="white"):
    h, w = frame_bgr.shape[:2]
    dpx = max(6, int(min(h, w) * 0.012))
    fpx = max(6, int(min(h, w) * 0.018))
    alpha = build_hand_mask(results, frame_bgr.shape, dpx, fpx)
    alpha3 = np.dstack([alpha, alpha, alpha])
    if mode == "blur":
        k = int(max(15, (min(h,w)//20)));  k += (k%2==0)
        bg = cv2.GaussianBlur(frame_bgr, (k, k), 0)
    else:
        bg = np.full_like(frame_bgr, 255)
    out = (frame_bgr * alpha3 + bg * (1 - alpha3)).astype(np.uint8)
    return out

# ---------- measurement ----------
def measure_fingers(annotated_bgr, results, scale_mm_per_px=None):
    h, w, _ = annotated_bgr.shape
    measurements = {}
    if not results or not results.multi_hand_landmarks:
        return annotated_bgr, measurements
    for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            annotated_bgr, hand_landmarks, mp_hands.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1)
        )
        for finger, ids in finger_joints.items():
            left_point  = hand_landmarks.landmark[ids[1]]
            right_point = hand_landmarks.landmark[ids[2]]
            dist_px, p1, p2 = euclidean_distance(left_point, right_point, w, h)
            dist_mm = dist_px * scale_mm_per_px if scale_mm_per_px else None
            us = diameter_to_us_ring_size(dist_mm) if isinstance(dist_mm,(int,float,np.floating)) else None
            eu = diameter_to_eu_size(dist_mm)      if isinstance(dist_mm,(int,float,np.floating)) else None
            measurements[finger] = {
                "pixels": float(dist_px),
                "diameter_mm": float(dist_mm) if dist_mm is not None else None,
                "us_ring_size": us,
                "eu_ring_size": eu
            }
            cv2.line(annotated_bgr, p1, p2, (0,255,0), 1)
            y0 = max(12, p1[1]-10)
            cv2.putText(annotated_bgr, f"{finger}: {dist_px:.1f}px", (p1[0], y0),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 1)
            if dist_mm is not None:
                cv2.putText(annotated_bgr, f"{dist_mm:.1f}mm  US:{us}  EU:{eu:.1f}",
                            (p1[0], y0-15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,0,0), 1)
            else:
                cv2.putText(annotated_bgr, "Calibrate to get US/EU",
                            (p1[0], y0-15), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (255,0,0), 1)
    return annotated_bgr, measurements

# ---------- Qt GUI ----------
class VideoLabel(QtWidgets.QLabel):
    clicked = QtCore.Signal(QtCore.QPoint)
    def mousePressEvent(self, e: QtGui.QMouseEvent):
        self.clicked.emit(e.position().toPoint())

class RingSizerApp(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("RingSizer — Hand Focus (Windows App)")
        self.resize(1100, 740)

        # UI
        self.video_label = VideoLabel()
        self.video_label.setStyleSheet("background:#111;")
        self.video_label.setAlignment(QtCore.Qt.AlignCenter)

        self.btn_start = QtWidgets.QPushButton("Start Camera")
        self.btn_stop  = QtWidgets.QPushButton("Stop Camera")
        self.btn_calib = QtWidgets.QPushButton("Calibrate (click 2 pts)")
        self.btn_save  = QtWidgets.QPushButton("Save PNG + JSON + PDF")
        self.btn_bg    = QtWidgets.QPushButton("BG: White")
        self.lbl_info  = QtWidgets.QLabel("Scale: (none)")
        self.lbl_info.setMinimumWidth(200)

        top = QtWidgets.QHBoxLayout()
        top.addWidget(self.btn_start)
        top.addWidget(self.btn_stop)
        top.addWidget(self.btn_calib)
        top.addWidget(self.btn_save)
        top.addWidget(self.btn_bg)
        top.addStretch(1)
        top.addWidget(self.lbl_info)

        central = QtWidgets.QWidget()
        lay = QtWidgets.QVBoxLayout(central)
        lay.addLayout(top)
        lay.addWidget(self.video_label, 1)
        self.setCentralWidget(central)

        # state
        self.cap = None
        self.timer = QtCore.QTimer(self)
        self.timer.timeout.connect(self.update_frame)

        self.bg_mode = "white"
        self.calibrating = False
        self.calib_pts_display = []  # points in label coords
        self.scale_mm_per_px = None
        self.reference_mm = None
        self.reference_pixels = None

        self.last_results = None
        self.last_annotated_bgr = None
        self.last_measurements = {}

        self.video_label.clicked.connect(self.label_clicked)
        self.btn_start.clicked.connect(self.start_cam)
        self.btn_stop.clicked.connect(self.stop_cam)
        self.btn_bg.clicked.connect(self.toggle_bg)
        self.btn_calib.clicked.connect(self.start_calibration)
        self.btn_save.clicked.connect(self.save_report)

        # Mediapipe context
        self.hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.7)

    # ----- Camera handling -----
    def try_open_camera(self):
        for idx in range(4):
            cap = cv2.VideoCapture(idx)
            if cap.isOpened():
                print(f"[INFO] Camera opened at index {idx}")
                return cap
            cap.release()
        return None

    def start_cam(self):
        if self.cap is not None: return
        self.cap = self.try_open_camera()
        if self.cap is None:
            QtWidgets.QMessageBox.warning(self, "Camera", "Could not open any camera (0..3). Close other apps and try again.")
            return
        self.timer.start(15)

    def stop_cam(self):
        self.timer.stop()
        if self.cap is not None:
            self.cap.release()
            self.cap = None

    # ----- UI actions -----
    def toggle_bg(self):
        self.bg_mode = "blur" if self.bg_mode == "white" else "white"
        self.btn_bg.setText(f"BG: {'Blur' if self.bg_mode=='blur' else 'White'}")

    def start_calibration(self):
        if self.last_annotated_bgr is None:
            QtWidgets.QMessageBox.information(self, "Calibration", "Start camera and show a known-length object in view.")
            return
        self.calibrating = True
        self.calib_pts_display = []
        QtWidgets.QMessageBox.information(self, "Calibration",
            "Click two points on a known-length object in the video.\n"
            "After two clicks you'll be prompted for the length in mm.")

    def label_clicked(self, pt: QtCore.QPoint):
        if not self.calibrating or self.last_annotated_bgr is None:
            return
        # map label point -> frame coords (account for aspect fit)
        lbl_w = self.video_label.width()
        lbl_h = self.video_label.height()
        frame_h, frame_w = self.last_annotated_bgr.shape[:2]
        scale = min(lbl_w / frame_w, lbl_h / frame_h)
        disp_w, disp_h = int(frame_w * scale), int(frame_h * scale)
        off_x = (lbl_w - disp_w) // 2
        off_y = (lbl_h - disp_h) // 2
        if not (off_x <= pt.x() <= off_x+disp_w and off_y <= pt.y() <= off_y+disp_h):
            return
        fx = (pt.x() - off_x) / scale
        fy = (pt.y() - off_y) / scale
        self.calib_pts_display.append((fx, fy))
        if len(self.calib_pts_display) == 2:
            (x1,y1), (x2,y2) = self.calib_pts_display
            pixels = float(math.hypot(x2-x1, y2-y1))
            mm, ok = QtWidgets.QInputDialog.getDouble(self, "Enter length",
                                                      f"Selected segment = {pixels:.2f} px\nEnter real length (mm):",
                                                      decimals=3, min=0.001, value=25.0)
            if ok and mm > 0:
                self.scale_mm_per_px = mm / pixels
                self.reference_mm = mm
                self.reference_pixels = pixels
                self.lbl_info.setText(f"Scale: {self.scale_mm_per_px:.5f} mm/px")
            self.calibrating = False
            self.calib_pts_display = []

    def save_report(self):
        if self.last_annotated_bgr is None or not self.last_measurements:
            QtWidgets.QMessageBox.information(self, "Save", "No hand/measurements yet.")
            return
        out = write_report_files(
            "reports",
            self.last_annotated_bgr,
            self.last_measurements,
            {
                "scale_mm_per_pixel": self.scale_mm_per_px,
                "reference_mm": self.reference_mm,
                "reference_pixels": self.reference_pixels,
                "notes": [f"GUI export (bg={self.bg_mode})"]
            }
        )
        QtWidgets.QMessageBox.information(self, "Saved",
            f"Image: {out['image']}\nJSON: {out['json']}\nPDF: {out['pdf']}")

    # ----- frame loop -----
    def update_frame(self):
        if self.cap is None: return
        ok, frame_bgr = self.cap.read()
        if not ok: return

        # landmarks
        rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb)

        # draw measurements on a copy
        annotated = frame_bgr.copy()
        annotated, meas = measure_fingers(annotated, results, self.scale_mm_per_px)

        # compose background
        composited = apply_hand_focus_effect(annotated, results, self.bg_mode)

        # keep last for saving
        self.last_results = results
        self.last_measurements = meas
        self.last_annotated_bgr = composited

        # draw calibration points for feedback
        if self.calibrating and len(self.calib_pts_display) > 0:
            for (fx, fy) in self.calib_pts_display:
                cv2.circle(self.last_annotated_bgr, (int(fx), int(fy)), 6, (0,255,255), -1)

        # show in QLabel
        rgb_show = cv2.cvtColor(self.last_annotated_bgr, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb_show.shape
        qimg = QtGui.QImage(rgb_show.data, w, h, ch*w, QtGui.QImage.Format_RGB888)
        pix = QtGui.QPixmap.fromImage(qimg)
        self.video_label.setPixmap(pix.scaled(
            self.video_label.size(), QtCore.Qt.KeepAspectRatio, QtCore.Qt.SmoothTransformation
        ))

    def closeEvent(self, e: QtGui.QCloseEvent):
        self.stop_cam()
        super().closeEvent(e)

def main():
    app = QtWidgets.QApplication(sys.argv)
    win = RingSizerApp()
    win.show()
    sys.exit(app.exec())

if __name__ == "__main__":
    main()
